# MIDS - w261 Machine Learning At Scale
__Course Lead:__ Dr James G. Shanahan (__email__ Jimi via  James.Shanahan _AT_ gmail.com)

## Assignment - HW5 Phase 2


---
__Name:__  Leslie Teo, Stan..., Sam Heung
__Class:__ MIDS w261 Winter 2018, Section 1     
__Email:__  lteo@iSchool.Berkeley.edu; lteo01@berkeley.edu     
__StudentId__  303218617    __End of StudentId__   
__Week:__   5.5

__NOTE:__ please replace `1234567` with your student id above      
__Due Time:__ HW is due the Thursday of the following week by 8AM (West coast time).

* __HW5 Phase 1__ 
This can be done on a local machine (with a unit test on the cloud such as Altiscale's PaaS or on AWS) and is due Thursday, Week 6 by 8AM (West coast time). It will primarily focus on building a unit/systems and for pairwise similarity calculations pipeline (for stripe documents)

* __HW5 Phase 2__ 
This will require the Altiscale cluster and will be due Thursday of the following week by 8AM (West coast time). 
The focus of  HW5 Phase 2  will be to scale up the unit/systems tests to the Google 5 gram corpus. 

# Datasets

For Phase 2 you will first use the small datasets from phase 1 to systems test your code in the cloud. Then you will test your code on 1 file and then 20 files before running the full (191 file) Google n-gram dataset.

__Small data for systems tests__

In [1]:
%%writefile atlas-boon-systems-test.txt
atlas boon	50	50	50
boon cava dipped	10	10	10
atlas dipped	15	15	15

Overwriting atlas-boon-systems-test.txt


In [2]:
%%writefile googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt
A BILL FOR ESTABLISHING RELIGIOUS	59	59	54
A Biography of General George	92	90	74
A Case Study in Government	102	102	78
A Case Study of Female	447	447	327
A Case Study of Limited	55	55	43
A Child's Christmas in Wales	1099	1061	866
A Circumstantial Narrative of the	62	62	50
A City by the Sea	62	60	49
A Collection of Fairy Tales	123	117	80
A Collection of Forms of	116	103	82

Overwriting googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt


SETUP: __Paths to Main data in HDFS on Altiscale AND OTHER SETTINGS__

In [3]:
TEST_1 = "/user/winegarj/data/1_test"
TEST_20 = "/user/winegarj/data/20_test"
FULL_DATA = "/user/winegarj/data/full" 
import os
USER = !whoami
USER = USER[0]
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

# Set - Up for Phase 2
Before you can run your simlarity analysis on the full Google n-gram dataset you should confirm that the code your wrote in Phase 1 works on the cloud. In the space below, copy the code for your three jobs from Phase 1 (`buildStripes.py`, `invertedIndex.py`, `similarity.py`) and rerun your  atlas-boon systems tests on Altiscale (i.e. ** the cloud**). NOTE: _you may end up modifying this code when you get to 5.7, that's fine._

### `buildStripes.py` Note: changed to `buildStripes_v2.py`

In [4]:
%%writefile buildStripes_v2.py
#!~/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import itertools

class MRbuildStripes(MRJob):
    SORT_VALUES = True
    
    def mapper(self, _, line):
        fields = line.lower().strip("\n").split("\t")
        words = fields[0].split(" ")
        occurrence_count = int(fields[1])
        for subset in itertools.combinations(sorted(set(words)), 2):
            yield subset[0], (subset[1], occurrence_count)
            yield subset[1], (subset[0], occurrence_count)
    
    def reducer(self, word, occurrence_counts):
        stripe = {}
        for other_word, occurrence_count in occurrence_counts:
            stripe[other_word] = stripe.get(other_word,0)+occurrence_count
        yield word, stripe

if __name__ == '__main__':
    MRbuildStripes.run()

Overwriting buildStripes_v2.py


### `invertedIndex.py` Note: changed to `invertedIndex_v2.py`

In [5]:
%%writefile invertedIndex_v2.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import division
import collections
import sys
import re
import json
import math
import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRinvertedIndex(MRJob):
    
    #START SUDENT CODE531_INV_INDEX
    
    SORT_VALUES = True
    
    def steps(self):
        
        JOBCONF_STEP = {
                'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                'mapreduce.partition.keycomparator.options': '-k1'
                }  
        return [
        MRStep(jobconf=JOBCONF_STEP, 
               mapper=self.mapper,
               reducer=self.reducer)
        ]
    
    def mapper(self, _, line):
        sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
        
        tokens = line.strip().split('\t')
        value_dict = json.loads(tokens[1])
        term_len = len(value_dict)
        
        for key in value_dict.keys():
            yield key, [tokens[0], term_len]

            
    def reducer(self, key, values):
        sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
        out = []
        for value_dict in values:
            value_dict[0] = value_dict[0].replace('"','')
            out.append(value_dict)
        
        yield key, out


  #END SUDENT CODE531_INV_INDEX
        
if __name__ == '__main__':
    MRinvertedIndex.run() 

Overwriting invertedIndex_v2.py


### `similarity.py` Note: changed to `similarity_v2.py`

In [6]:
%%writefile similarity_v2.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import division
import sys
import collections
import re
import json
import math
import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.protocol import JSONProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRsimilarity(MRJob):
    SORT_VALUES = True
    OUTPUT_PROTOCOL = RawProtocol
    
    def steps(self):
        
        JOBCONF_STEP_2 = {
                'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                'mapreduce.partition.keycomparator.options': '-k1,1nr',
                "mapreduce.job.reduces": "1",
                "SORT_VALUES":True
                }  
        JOBCONF_STEP_1 = {
                "mapreduce.job.reduces": "64",
                "mapreduce.job.maps": "64",
                } 
        return [
        MRStep(jobconf=JOBCONF_STEP_1,
               mapper=self.mapper_pair_sim,
               reducer=self.reducer_pair_sim
            ),
        MRStep(jobconf=JOBCONF_STEP_2, 
               reducer=self.reducer_pair_sim2
            )
        ]

    def mapper_pair_sim(self, _, line):     
        sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
        line = line.strip()
        index, posting = line.split('\t')
        posting = json.loads(posting)
        posting = dict(posting)
        for docs in itertools.combinations(sorted(posting.keys()), 2):
            yield (docs, posting[docs[0]], posting[docs[1]]), 1
        

    def reducer_pair_sim(self, key, values):
        sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
        total = sum(values)
        cosine = total/(np.sqrt(key[1])*np.sqrt(key[2]))
        jacard = total/(key[1]+key[2]-total)
        overlap = total/min(key[1],key[2])
        dice = 2*total/(key[1]+key[2])
        yield np.mean([cosine, jacard, overlap, dice]), (key[0][0]+' - '+key[0][1],cosine,jacard,overlap,dice)

    def reducer_pair_sim2(self, key, values):
        sys.stderr.write("reporter:counter:Intermediate Reducer Counters,Calls,1\n")
        for value in values:
            yield str(key), json.dumps(value)

if __name__ == '__main__':
    MRsimilarity.run()

Overwriting similarity_v2.py


#### atlas-boon systems test

In [ ]:
!python buildStripes_v2.py \
    -r local atlas-boon-systems-test.txt

In [ ]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!python buildStripes_v2.py \
        -r hadoop atlas-boon-systems-test.txt \
        --output-dir={OUTPUT_PATH} \
        --no-output
    

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/*

In [ ]:
# Save into file for processing
!hadoop fs -cat {OUTPUT_PATH}/* > test_stripes_1

In [ ]:
# Testing inverted index
!python invertedIndex_v2.py \
    -r local test_stripes_1

In [ ]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!python invertedIndex_v2.py \
        -r hadoop test_stripes_1 \
        --output-dir={OUTPUT_PATH} \
        --no-output
    

In [ ]:
# Save into file for processing
!hadoop fs -cat {OUTPUT_PATH}/* > test_index_1
!cat test_index_1

In [ ]:
##########################################################
# Pretty print systems tests for generating Inverted Index
##########################################################

import json

for i in range(1,2):
    print "—"*100
    print "Systems test ",i," - Inverted Index"
    print "—"*100  
    with open("test_index_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            word,stripe = line.split("\t")
            stripe = json.loads(stripe)
            stripe.extend([["",""] for _ in xrange(3 - len(stripe))])

            print "{0:>16} |{1:>16} |{2:>16} |{3:>16}".format((word), 
                stripe[0][0]+" "+str(stripe[0][1]), stripe[1][0]+" "+str(stripe[1][1]), stripe[2][0]+" "+str(stripe[2][1]))

In [ ]:
# Testing similarity metrics
!python similarity_v2.py \
    -r local test_index_1

In [ ]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!python similarity_v2.py \
        -r hadoop test_index_1 \
        --output-dir={OUTPUT_PATH} \
        --no-output
    

In [ ]:
# Save into file for processing
!hadoop fs -cat {OUTPUT_PATH}/* > test_similarities_1
!cat test_similarities_1

In [ ]:
############################################
# Pretty print systems tests
# Note: adjust print formatting if you need to
############################################

import json
for i in range(1,2):
    print '—'*110
    print "Systems test ",i," - Similarity measures"
    print '—'*110
    print "{0:>15} |{1:>15} |{2:>15} |{3:>15} |{4:>15} |{5:>15}".format(
    "average", "pair", "cosine", "jaccard", "overlap", "dice")
    print '-'*110

    with open("test_similarities_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            avg,stripe = line.split("\t")
            stripe = json.loads(stripe)
            
            print "{0:>15f} |{1:>15} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(float(avg),
                stripe[0], float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]))



#### 10-line systems test

In [ ]:
# Build Stripes

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python buildStripes_v2.py \
        -r hadoop googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt \
        --output-dir={OUTPUT_PATH} \
        --no-output

!hadoop fs -cat {OUTPUT_PATH}/* > test_stripes_2



In [ ]:
!cat test_stripes_2

In [ ]:
# Build Inverted Index

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python invertedIndex_v2.py \
        -r hadoop test_stripes_2 \
        --output-dir={OUTPUT_PATH} \
        --no-output
        
!hadoop fs -cat {OUTPUT_PATH}/* > test_index_2

In [ ]:
# Calculate Similarity

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python similarity_v2.py \
        -r hadoop test_index_2 \
        --output-dir={OUTPUT_PATH} \
        --no-output
        
!hadoop fs -cat {OUTPUT_PATH}/* > test_similarities_2 

In [ ]:
############################################
# Pretty print systems tests
# Note: adjust print formatting if you need to
############################################

import json
for i in range(1,3):
    print '—'*110
    print "Systems test ",i," - Similarity measures"
    print '—'*110
    print "{0:>15} |{1:>15} |{2:>15} |{3:>15} |{4:>15} |{5:>15}".format(
    "average", "pair", "cosine", "jaccard", "overlap", "dice")
    print '-'*110

    with open("test_similarities_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            avg,stripe = line.split("\t")
            stripe = json.loads(stripe)
            
            print "{0:>15f} |{1:>15} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(float(avg),
                stripe[0], float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]))

# HW5.6 -Google n-grams EDA

Do some EDA on this dataset using mrjob, e.g., 

- A. Longest 5-gram (number of characters)
- B. Top 10 most frequent words (please use the count information), i.e., unigrams
- C. 20 Most/Least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency 
- D. Distribution of 5-gram sizes (character length).  E.g., count (using the count field) up how many times a 5-gram of 50 characters shows up. Plot the data graphically using a histogram.

### HW5.6.1 - A. Longest 5-gram (number of characters)

In [ ]:
%%writefile longest5gram.py
#!/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-

import re
from datetime import datetime
import sys

import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class longest5gram(MRJob):
#     SORT_VALUES = True
    def mapper(self, _, line):
        fields = line.strip("\n").split("\t")
        yield len(fields[0]), fields[0]
    
    def reducer_init(self):
        self.longest_ngrams = []
        self.longest_size = 0
        
    def reducer(self, key, values):
        if int(key)> self.longest_size:
            self.longest_size = int(key)
            self.longest_ngrams = list(values)
        elif int(key) == self.longest_size:
            self.longest_ngrams = list(self.longest_ngrams)+list(values)
            
    def reducer_final(self):
        yield self.longest_size, ";".join(list(self.longest_ngrams))
    
    def reducer_2_init(self):
        self.longest_2_ngrams = []
        self.longest_2_size = 0
    
    def reducer_2(self, key, values):
        if int(key)> self.longest_2_size:
            self.longest_2_size = int(key)
            self.longest_2_ngrams = list(values)
        elif int(key) == self.longest_2_size:
            self.longest_2_ngrams = list(self.longest_2_ngrams)+list(values)
            
    def reducer_2_final(self):
        yield self.longest_2_size, ";".join(list(self.longest_2_ngrams))
        
    def steps(self):
        return [
            MRStep(
                mapper = self.mapper,
                reducer_init = self.reducer_init,
                reducer_final = self.reducer_final,
                reducer = self.reducer,
                jobconf={
                    "mapreduce.job.reduces": "32",
                    "stream.num.map.output.key.fields": 1,
                    "mapreduce.job.output.key.comparator.class" : "org.apache.hadoop.mapred.lib.KeyFieldBasedComparator",
                    "mapreduce.partition.keycomparator.options":"-k1,1nr",
                }
            ),
            MRStep(
                reducer_init = self.reducer_2_init,
                reducer_final = self.reducer_2_final,
                reducer = self.reducer_2,
                jobconf={
                    "mapreduce.job.reduces": "1"
                }
            )
        ]
    
if __name__ == '__main__':
    start_time = datetime.now()
    longest5gram.run()
    end_time = datetime.now()
    elapsed_time = end_time - start_time
    sys.stderr.write(str(elapsed_time))

__On test data set:__

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'longest_ngram_10lines')
!hadoop fs -rm -r {OUTPUT_PATH}
!time python longest5gram.py \
        -r hadoop googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/*

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'longest_ngram_test1')
!hadoop fs -rm -r {OUTPUT_PATH}
!time python longest5gram.py \
        -r hadoop hdfs://{TEST_1} \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hdfs dfs -cat {OUTPUT_PATH}/*

__ On the 20 files dataset: __

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'longest_ngram_test20')
!hadoop fs -rm -r {OUTPUT_PATH}
!time python longest5gram.py \
        -r hadoop hdfs://{TEST_20} \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hdfs dfs -cat {OUTPUT_PATH}/*

__On full data set:__

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'longest_full')
!hadoop fs -rm -r {OUTPUT_PATH}
!time python longest5gram.py \
        -r hadoop hdfs://{FULL_DATA} \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/*

### Longest 5grams MR stats

    ec2_instance_type: m3.xlarge
    num_ec2_instances: 15

__Step 1:__  

    RUNNING for 107.0s ~= 2 minutes  
    Reduce tasks = 16 
    
__Step 2:__   

    RUNNING for 108.8s ~= 2 minutes
    Reduce tasks = 1  

### HW5.6.1 - B. Top 10 most frequent words 

In [ ]:
%%writefile mostFrequentWords.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

import re

import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class mostFrequentWords(MRJob):
        
    # START STUDENT CODE 5.6.1.B
    
    SORT_VALUES = True
    
    def steps(self):
        
        
        JOBCONF_STEP1 = {'mapreduce.job.reduces': '10',
                        }
        
        JOBCONF_STEP2 = {
                'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                'stream.num.map.output.key.fields':'2',
                'stream.map.output.field.separator':'\t',    
                'mapreduce.partition.keycomparator.options': '-k1,1nr',
                'mapreduce.job.reduces': '1', 
                }  
        return [
        MRStep(#jobconf=JOBCONF_STEP1, 
               mapper=self.mapper,
               combiner=self.combiner,
               reducer=self.reducer,
              ),
        MRStep(jobconf=JOBCONF_STEP2,
              mapper=self.mapper2,
              ),
        ]

    def mapper(self, _, line):
        words = re.findall(r'[a-z\']+', line.lower())
        for word in words:
            yield word, 1
                         
    def combiner(self, word, counts):
        yield word, sum(counts)
            
    def reducer(self, word, counts):
        yield word, sum(counts)
        
    def mapper2(self, word, counts):
        yield counts, word
    
    # END STUDENT CODE 5.6.1.B
        
        
if __name__ == '__main__':
    mostFrequentWords.run()

__On the test data set:__

In [ ]:
!python mostFrequentWords.py \
    -r local googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt

In [ ]:
# Find top 10 most frequent words
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!python mostFrequentWords.py \
        -r hadoop hdfs://{TEST_1}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output


In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* | head -n 10

__ On the 20 files dataset: __

In [ ]:
# Find top 10 most frequent word

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!python mostFrequentWords.py \
        -r hadoop hdfs://{TEST_20}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output


In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* | head -n 10

__On the full data set:__

In [ ]:
# Find top 10 most frequent word

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!python mostFrequentWords.py \
        -r hadoop hdfs://{FULL_DATA}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* | head -n 10000 > test.output

In [ ]:
!cat test.output | head -n 100
!cat test.output | tail -n 10

**Version that excludes stop words**

In [ ]:
%%writefile mostFrequentWords_v2.py
#!/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-

import re

import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class mostFrequentWords(MRJob):
        
    # START STUDENT CODE 5.6.1.B
    
    SORT_VALUES = True
    
    def steps(self):
        
        
        JOBCONF_STEP1 = {'mapreduce.job.reduces': '10',
                        }
        
        JOBCONF_STEP2 = {
                'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                'stream.num.map.output.key.fields':2,
                'stream.map.output.field.separator':'\t',    
                'mapreduce.partition.keycomparator.options': '-k1,1nr',
                'mapreduce.job.reduces': '1', 
                }  
        return [
        MRStep(jobconf=JOBCONF_STEP1, 
               mapper_init=self.mapper_init,
               mapper=self.mapper,
               combiner=self.combiner,
               reducer=self.reducer,
              ),
        MRStep(jobconf=JOBCONF_STEP2,
              mapper=self.mapper2,
              ),
        ]
    
    def mapper_init(self):
        self.stopwords =  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 'just', 
              'don', 'should', 'now']

    def mapper(self, _, line):
        words = re.findall(r'[a-z\']+', line.lower())
        for word in words:
            if word not in self.stopwords:
                yield word, 1
                         
    def combiner(self, word, counts):
        yield word, sum(counts)
            
    def reducer(self, word, counts):
        yield word, sum(counts)
        
    def mapper2(self, word, counts):
        yield counts, word
    
    # END STUDENT CODE 5.6.1.B
        
        
if __name__ == '__main__':
    mostFrequentWords.run()

In [ ]:
!python mostFrequentWords_v2.py \
    -r local googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt

In [ ]:
# Find top 10 most frequent words
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!python mostFrequentWords_v2.py \
        -r hadoop hdfs://{TEST_1}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output


In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* | head -n 100

### Most frequent words MR stats
    
    ec2_instance_type: m3.xlarge
    num_ec2_instances: 15
    
__Step 1:__   

    RUNNING for 590.7s ~= 10 minutes   
    Launched map tasks=191  
    Launched reduce tasks=57   

__Step 2:__  

    RUNNING for 76.6s   
    Launched map tasks=110
    Launched reduce tasks=16  


### HW5.6.1 - C. 20 Most/Least densely appearing words 

In [ ]:
%%writefile mostLeastDenseWords.py
#!/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-
from __future__ import division
import re
import numpy as np
import mrjob
import json
import sys
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class mostLeastDenseWords(MRJob):
    
    # START STUDENT CODE 5.6.1.C
    OUTPUT_PROTOCOL = RawProtocol
    SORT_VALUES = True
    total_page_count = 0 
    
    def steps(self):  
        JOBCONF_STEP1 = {'mapreduce.job.reduces': '10'}
        
        JOBCONF_STEP2 = {
                'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                'stream.num.map.output.key.fields':'2',
                'stream.map.output.field.separator':'\t',    
                'mapreduce.partition.keycomparator.options': '-k2,2nr',
                'mapreduce.job.reduces': '1', 
                }  
        
        return [MRStep(jobconf=JOBCONF_STEP1, 
                mapper=self.mapper,
                reducer=self.reducer
                ),
                MRStep(jobconf=JOBCONF_STEP2,
                reducer=self.reducer_output)
               ]

    def mapper(self, _, line):
        data = line.split("\t")
        words = data[0].lower().split()
        count = int(data[1])
        page_count = int(data[2])
        
        for w in words:
            yield w, count

        yield "!Total", page_count
            
    def reducer(self, key, data):
        yield key, sum(data)

    def reducer_output(self, key, data):
        if key == "!Total":
            self.total_page_count = sum(data)
        else:
            yield key, str(sum(data)/self.total_page_count)
            
    # END STUDENT CODE 5.6.1.C
        
if __name__ == '__main__':
    mostLeastDenseWords.run()

__On the test data set:__

In [ ]:
!python mostLeastDenseWords.py \
    -r local googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt

In [ ]:
# Density for 1 file

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}


!time python mostLeastDenseWords.py \
        -r hadoop hdfs://{TEST_1}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output
        

In [ ]:
!hdfs dfs -cat {OUTPUT_PATH}/*

__ On the 20 files dataset: __

__On the full data set:__

### Word density MR stats

    ec2_instance_type: m3.xlarge
    num_ec2_instances: 15
    
__Step 1:__ 

    RUNNING for 649.2s  ~= 10 minutes      
    Launched map tasks=190   
    Launched reduce tasks=57     

__Step 2:__  

    RUNNING for 74.4s  ~= 1 minute    
    Launched map tasks=110   
    Launched reduce tasks=20   

### HW5.6.1 - D. Distribution of 5-gram sizes (character length)

In [ ]:
%%writefile distribution.py
#!/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-

import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class distribution(MRJob):
    
    #### TODO: divide the counts by 1000s to make the graph more readable
    #### TODO: split the lengths into buckets <10, <25, <50, <75, <100
    def mapper(self, _, line):
        fields = line.strip("\n").split("\t")
        yield len(fields[0]), int(fields[1])
    
    def combiner(self,length, counts):
        yield length, sum(counts)
        
    def reducer(self,length, counts):
        yield length, sum(counts)
    
    def reducer_sort(self, key, values):
        yield key, list(values)[0]
        
    def steps(self):
        return [
            MRStep(
                mapper = self.mapper,
                combiner = self.combiner,
                reducer = self.reducer,
                jobconf = {
                    "mapreduce.job.reduces": "4",
                }
            ),
            MRStep(
                reducer = self.reducer_sort,
                jobconf = {
                    "SORT_VALUES":True,
                    "mapreduce.job.reduces": "1",
                    "stream.num.map.output.key.fields": 1,
                    "mapreduce.job.output.key.comparator.class" : "org.apache.hadoop.mapred.lib.KeyFieldBasedComparator",
                    "mapreduce.partition.keycomparator.options":"-k1,1nr",
                }
            )
        ]
    
if __name__ == '__main__':
    distribution.run()

__On the test data set:__

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'distributions_10lines')
!hadoop fs -rm -r {OUTPUT_PATH}
!time python distribution.py \
        -r hadoop googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* > distributions_10lines.txt

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'distributions_1file')
!hadoop fs -rm -r {OUTPUT_PATH}
!python distribution.py \
        -r hadoop hdfs://{TEST_1} \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* > distributions_1file.txt

__ On the 20 files dataset: __

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'distributions_20files')
!hadoop fs -rm -r {OUTPUT_PATH}
!time python distribution.py \
        -r hadoop hdfs://{TEST_20} \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* > distributions_20files.txt

__On the full data set:__

In [ ]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'distributions_full')
!hadoop fs -rm -r {OUTPUT_PATH}
!time python distribution.py \
        -r hadoop hdfs://{FULL_DATA} \
        --output-dir={OUTPUT_PATH} \
        --no-output

In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* > distributions_full.txt

### Distribution MRJob stats

__Step 1:__ 

    RUNNING for 157.8s ~= 2.6 minutes  
    Launched map tasks=191  
    Launched reduce tasks=16   
    
__Step 2:__  

    RUNNING for 115.0s ~= 2 minutes   
    Launched map tasks=139
	Launched reduce tasks=1  

In [ ]:
%matplotlib inline
import numpy as np
import pylab as pl

results_A = []
for line in open("distributions_10lines.txt").readlines():
    line = line.strip()
    X,Y = line.split("\t")
    results_A.append([int(X),int(Y)])

items = (np.array(results_A)[::-1].T)
fig = pl.figure(figsize=(17,7))
ax = pl.subplot(111)
width=0.8
ax.bar(range(len(items[0])), items[1], width=width)
ax.set_xticks(np.arange(len(items[0])) + width/2)
ax.set_xticklabels(items[0], rotation=90)



pl.title("Distributions of 5 Gram lengths using 10-line sample")
pl.show()

In [ ]:
%matplotlib inline
import numpy as np
import pylab as pl

results_A = []
for line in open("distributions_1file.txt").readlines():
    line = line.strip()
    X,Y = line.split("\t")
    results_A.append([int(X),int(Y)])

items = (np.array(results_A)[::-1].T)
fig = pl.figure(figsize=(17,7))
ax = pl.subplot(111)
width=0.8
ax.bar(range(len(items[0])), items[1], width=width)
ax.set_xticks(np.arange(len(items[0])) + width/2)
ax.set_xticklabels(items[0], rotation=90)



pl.title("Distributions of 5 Gram lengths using 1 file")
pl.show()

In [ ]:
%matplotlib inline
import numpy as np
import pylab as pl

results_A = []
for line in open("distributions_20files.txt").readlines():
    line = line.strip()
    X,Y = line.split("\t")
    results_A.append([int(X),int(Y)])

items = (np.array(results_A)[::-1].T)
fig = pl.figure(figsize=(17,7))
ax = pl.subplot(111)
width=0.8
ax.bar(range(len(items[0])), items[1], width=width)
ax.set_xticks(np.arange(len(items[0])) + width/2)
ax.set_xticklabels(items[0], rotation=90)



pl.title("Distributions of 5 Gram lengths using 20 files")
pl.show()

In [ ]:
%matplotlib inline
import numpy as np
import pylab as pl

results_A = []
for line in open("distributions_full.txt").readlines():
    line = line.strip()
    X,Y = line.split("\t")
    results_A.append([int(X),int(Y)])

items = (np.array(results_A)[::-1].T)
fig = pl.figure(figsize=(17,7))
ax = pl.subplot(111)
width=0.8
ax.bar(range(len(items[0])), items[1], width=width)
ax.set_xticks(np.arange(len(items[0])) + width/2)
ax.set_xticklabels(items[0], rotation=90)



pl.title("Distributions of 5 Gram lengths using all files")
pl.show()

### HW5.6.2 - OPTIONAL: log-log plots (PHASE 2)

Plot the log-log plot of the frequency distributuion of unigrams. Does it follow power law distribution?

For more background see:
- https://en.wikipedia.org/wiki/Log%E2%80%93log_plot
- https://en.wikipedia.org/wiki/Power_law

# HW5.7 - Synonym detection over 2Gig of Data with extra Preprocessing steps (HW5.3-5 plus some preprocessing)   

For the remainder of this assignment please feel free to eliminate stop words from your analysis (see stopWords in the cell below)

__A large subset of the Google n-grams dataset as was described above__

For each HW 5.6 - 5.7.1 Please unit test and system test your code with respect 
to SYSTEMS TEST DATASET and show the results. 
Please compute the expected answer by hand and show your hand calculations for the 
SYSTEMS TEST DATASET. Then show the results you get with your system.

In this part of the assignment we will focus on developing methods for detecting synonyms, using the Google 5-grams dataset. At a high level:


1. remove stopwords
2. get 10,000 most frequent
3. get 1000 (9001-10000) features
3. build stripes

To accomplish this you must script two main tasks using MRJob:


__TASK (1)__ Build stripes for the most frequent 10,000 words using cooccurence information based on
the words ranked from 9001,-10,000 as a basis/vocabulary (drop stopword-like terms),
and output to a file in your bucket on s3 (bigram analysis, though the words are non-contiguous).


__TASK (2)__ Using two (symmetric) comparison methods of your choice 
(e.g., correlations, distances, similarities), pairwise compare 
all stripes (vectors), and output to a file in your bucket on s3.


For this task you will have to determine a method of comparison.
Here are a few that you might consider:

- Jaccard
- Cosine similarity
- Spearman correlation
- Euclidean distance
- Taxicab (Manhattan) distance
- Shortest path graph distance (a graph, because our data is symmetric!)
- Pearson correlation
- Kendall correlation

However, be cautioned that some comparison methods are more difficult to
parallelize than others, and do not perform more associations than is necessary, 
since your choice of association will be symmetric.

Please report the size of the cluster used and the amount of time it takes to run for the index construction task and for the synonym calculation task. How many pairs need to be processed (HINT: use the posting list length to calculate directly)? Report your  Cluster configuration!

In [ ]:
stopwords =  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 'just', 
              'don', 'should', 'now']

__STEP 1: Code and Steps for Preprocessing__

In [ ]:
# Remove stop words, get 10,000 most frequent words

# Find top 10,000 most frequent words
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'ten_thousand_1')
!hadoop fs -rm -r {OUTPUT_PATH}

!python mostFrequentWords_v2.py \
        -r hadoop hdfs://{TEST_1}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output


In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* | head -n 10000 > ten_thousand_1.dat

In [ ]:
# Find top 10,000 most frequent words
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'ten_thousand_20')
!hadoop fs -rm -r {OUTPUT_PATH}

!python mostFrequentWords_v2.py \
        -r hadoop hdfs://{TEST_20}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output


In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* | head -n 10000 > ten_thousand_20.dat

In [ ]:
!cat ten_thousand_20.dat | head
!cat ten_thousand_20.dat | tail

In [ ]:
# Find top 10,000 most frequent words
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'ten_thousand_FULL')
!hadoop fs -rm -r {OUTPUT_PATH}

!python mostFrequentWords_v2.py \
        -r hadoop hdfs://{FULL_DATA}/* \
        --output-dir={OUTPUT_PATH} \
        --no-output


In [ ]:
!hadoop fs -cat {OUTPUT_PATH}/* | head -n 10000 > ten_thousand_FULL.dat

In [ ]:
!cat ten_thousand_FULL.dat | head
!cat ten_thousand_FULL.dat | tail

In [ ]:
# Also create features
!cat ten_thousand_1.dat |tail -n 999 > features_1.dat
!cat ten_thousand_20.dat |tail -n 999 > features_20.dat
!cat ten_thousand_FULL.dat |tail -n 999 > features_FULL.dat

**We now have files with 10K words => `ten_thousand_*.dat` AND
features with words from 9,001 to 10,000 in => `features_*.dat` **

In [92]:
### Some extra preprocessing to load files faster
import json
files = ['features_1','features_20', 'features_FULL', 'ten_thousand_1', 'ten_thousand_20','ten_thousand_FULL']
for fileName in files:
    with open(fileName+'.dat') as f:
        words = []
        for line in f:
            word = line.split("\t")[1].strip('"')
            words.append(word)
    with open(fileName+'.json', 'w') as outfile:
        json.dump(words, outfile)

__STEP 2: MODIFY BUILD STRIPES TO ONLY INCLUDE 10K words with 1K as FEATURES__

In [8]:
%%writefile buildStripes_stopwords_1.py
#!~/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import itertools

class MRbuildStripes(MRJob):
    
    SORT_VALUES = True
    
    def steps(self):
        return [
            MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper,
                reducer=self.reducer,
                jobconf = {
                    "mapreduce.job.reduces": "64",
                    "mapreduce.job.maps": "64",
#                     "SORT_VALUES":True
                }
            ),
            MRStep(
                reducer=self.reducer_2,
                jobconf = {
                    "mapreduce.job.reduces": "1",
                    "SORT_VALUES":True
                }
            )
            
        ]
    
    def mapper_init(self):
        self.idx = 9001  # To define when feature set starts
        self.filename = 'ten_thousand_1.json'
        
        self.top_words = []
        self.features = []
        #with open('features_20.json', 'r') as infile:
        #    self.features = json.loads(infile.read())
        with open(self.filename, 'r') as infile:
            self.top_words = json.loads(infile.read())
            self.features =self.top_words[self.idx:]

    def mapper(self, _, line):
        fields = line.lower().strip("\n").split("\t")
        words = fields[0].split(" ")
        occurrence_count = int(fields[1])
        filtered_words = [word.decode('utf-8', 'ignore') for word in words if word.decode('utf-8', 'ignore') in self.top_words]
        for subset in itertools.combinations(sorted(set(filtered_words)), 2):
            if subset[0] in self.top_words and subset[1] in self.features:
                yield subset[0], (subset[1], occurrence_count)
            if subset[1] in self.top_words and subset[0] in self.features:
                yield subset[1], (subset[0], occurrence_count)
    
    def reducer(self, word, occurrence_counts):
        stripe = {}
        for other_word, occurrence_count in occurrence_counts:
            stripe[other_word] = stripe.get(other_word,0)+occurrence_count
        yield word, stripe
    
    def reducer_2(self, key, values):
        yield str(key), list(values)[0]
        
if __name__ == '__main__':
    MRbuildStripes.run()

Overwriting buildStripes_stopwords_1.py


### HW5.7.1 Running on 1 file

In [12]:
TEST_1 = "/user/winegarj/data/1_test"
TEST_20 = "/user/winegarj/data/20_test"
FULL_DATA = "/user/winegarj/data/full" 

import os
USER = !whoami
USER = USER[0]
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

In [13]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python buildStripes_stopwords_1.py \
        -r hadoop hdfs://{TEST_1}/* \
        --file ten_thousand_1.json  \
        --output-dir={OUTPUT_PATH} \
        --no-output

    

18/03/06 15:35:59 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/buildStripes_stopwords_1.lteo01.20180306.153600.339221
Copying local files to hdfs:///user/lteo01/tmp/mrjob/buildStripes_stopwords_1.lteo01.20180306.153600.339221/files/...
Running step 1 of 2...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob3079440485959886461.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altisca

Removing temp directory /tmp/buildStripes_stopwords_1.lteo01.20180306.153600.339221...

real	2m49.371s
user	0m50.082s
sys	0m2.459s


In [ ]:
####NOT USED, BUT KEEP JUST IN CASE
import os
from os import listdir
from os.path import isfile, join

def totalOrderSort(myPath, outFileName):
    wordsFiles = {}
    words = []
    for f in listdir(myPath):
        if isfile(join(myPath,f)) and "part" in f:
            with open(join(myPath,f)) as openFile:
                word = f.readline().split("\t")[0]
                words.append(word)
                wordsFiles[word]= join(mypath,f)
    print wordsFiles
    print words

In [14]:
# Look at data
# !hadoop fs -cat {OUTPUT_PATH}/*
# Save into file for processing
!hadoop fs -cat {OUTPUT_PATH}/* > google_stripes_1

In [15]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python invertedIndex_v2.py \
        -r hadoop google_stripes_1 \
        --output-dir={OUTPUT_PATH} \
        --no-output


18/03/06 15:38:53 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/invertedIndex_s.lteo01.20180306.153854.119645
Copying local files to hdfs:///user/lteo01/tmp/mrjob/invertedIndex_s.lteo01.20180306.153854.119645/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob198206904659061463.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/t

In [17]:
# Save into file for processing
!hadoop fs -cat {OUTPUT_PATH}/* > google_index_1
!cat google_index_1 | head

"ab"	[["cannot", 6], ["cd", 1], ["chord", 2], ["equal", 4], ["lines", 2], ["piece", 3], ["quantity", 3], ["university", 12]]
"abnormalities"	[["associated", 3], ["b", 3], ["found", 17], ["indicate", 2], ["may", 55], ["organs", 1], ["patients", 12], ["reproductive", 1], ["result", 2], ["show", 5], ["usually", 3], ["vitamin", 1]]
"absolve"	[["citizens", 1], ["english", 14], ["lord", 7], ["may", 55], ["obligations", 1], ["right", 13], ["thee", 2], ["would", 47]]
"absorbing"	[["beautiful", 4], ["become", 12], ["capable", 5], ["case", 11], ["cost", 2], ["far", 15], ["france", 1], ["interest", 10], ["light", 6], ["may", 55], ["molecules", 1], ["still", 8]]
"abundantly"	[["apparently", 2], ["become", 12], ["clear", 1], ["consumption", 1], ["exemplified", 1], ["found", 17], ["naked", 1], ["supplied", 2]]
"abyss"	[["distress", 1], ["lead", 7], ["nothing", 6], ["occupied", 1], ["previous", 4], ["separates", 2], ["ultimately", 2], ["us", 21]]
"ac"	[["angle", 4], ["circuit", 2], ["components", 1],

In [ ]:
##########################################################
# Pretty print systems tests for generating Inverted Index
##########################################################

import json

for i in range(1,2):
    print "—"*100
    print "Systems test ",i," - Inverted Index"
    print "—"*100  
    with open("google_index_"+str(i)+"","r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            word,stripe = line.split("\t")
            stripe = json.loads(stripe)
            stripe.extend([["",""] for _ in xrange(3 - len(stripe))])

            print "{0:>16} |{1:>16} |{2:>16} |{3:>16}".format((word), 
                stripe[0][0]+" "+str(stripe[0][1]), stripe[1][0]+" "+str(stripe[1][1]), stripe[2][0]+" "+str(stripe[2][1]))

In [18]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python similarity_v2.py \
        -r hadoop google_index_1 \
        --output-dir={OUTPUT_PATH} \
        --no-output
            

18/03/06 15:45:11 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/similarity_v2.lteo01.20180306.154512.772830
Copying local files to hdfs:///user/lteo01/tmp/mrjob/similarity_v2.lteo01.20180306.154512.772830/files/...
Running step 1 of 2...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob6359915834628725457.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/time

In [19]:
# Save into file for processing
!hadoop fs -cat {OUTPUT_PATH}/* > google_similarities_1
!cat google_similarities_1 | head

1.0	["conspicuous - temper", 1.0, 1.0, 1.0, 1.0]
1.0	["honest - mentioned", 1.0, 1.0, 1.0, 1.0]
1.0	["conscience - estate", 1.0, 1.0, 1.0, 1.0]
1.0	["susceptible - undertook", 1.0, 1.0, 1.0, 1.0]
1.0	["check - migration", 1.0, 1.0, 1.0, 1.0]
1.0	["citizens - obligations", 1.0, 1.0, 1.0, 1.0]
1.0	["defect - edge", 1.0, 1.0, 1.0, 1.0]
1.0	["translation - varied", 1.0, 1.0, 1.0, 1.0]
1.0	["retina - stigma", 1.0, 1.0, 1.0, 1.0]
1.0	["ad - illinois", 1.0, 1.0, 1.0, 1.0]
cat: write error: Broken pipe


In [ ]:
############################################
# Pretty print systems tests
# Note: adjust print formatting if you need to
############################################

import json
for i in range(1,2):
    print '—'*110
    print "Systems test ",i," - Similarity measures"
    print '—'*110
    print "{0:>21} | {1:>15} |{2:>15} | {3:>15} | {4:>15} | {5:>15}".format("pair", 
        "cosine", "jaccard", "overlap", "dice", "average")
    print '-'*110

    with open("google_similarities_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            avg,stripe = line.split("\t")
            stripe = json.loads(stripe)
            
            print "{0:>21} | {1:>15f} |{2:>15f} |{3:>15f} | {4:>15f} | {5:>15f} ".format(stripe[0], 
                float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]), float(avg))

In [ ]:
# ADD CELLS AS NEEDED

### HW5.7.2 Running on 20 test files

In [23]:
%%writefile buildStripes_stopwords_20.py
#!~/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import itertools

class MRbuildStripes(MRJob):
    
    SORT_VALUES = True
    
    def steps(self):
        return [
            MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper,
                reducer=self.reducer,
                jobconf = {
                    "mapreduce.job.reduces": "64",
                    "mapreduce.job.maps": "64",
#                     "SORT_VALUES":True
                }
            ),
            MRStep(
                reducer=self.reducer_2,
                jobconf = {
                    "mapreduce.job.reduces": "1",
                    "SORT_VALUES":True
                }
            )
            
        ]
    
    def mapper_init(self):
        self.idx = 9001  # To define when feature set starts
        self.filename = 'ten_thousand_20.json'
        
        self.top_words = []
        self.features = []
        #with open('features_20.json', 'r') as infile:
        #    self.features = json.loads(infile.read())
        with open(self.filename, 'r') as infile:
            self.top_words = json.loads(infile.read())
            self.features =self.top_words[self.idx:]

    def mapper(self, _, line):
        fields = line.lower().strip("\n").split("\t")
        words = fields[0].split(" ")
        occurrence_count = int(fields[1])
        filtered_words = [word.decode('utf-8', 'ignore') for word in words if word.decode('utf-8', 'ignore') in self.top_words]
        for subset in itertools.combinations(sorted(set(filtered_words)), 2):
            if subset[0] in self.top_words and subset[1] in self.features:
                yield subset[0], (subset[1], occurrence_count)
            if subset[1] in self.top_words and subset[0] in self.features:
                yield subset[1], (subset[0], occurrence_count)
    
    def reducer(self, word, occurrence_counts):
        stripe = {}
        for other_word, occurrence_count in occurrence_counts:
            stripe[other_word] = stripe.get(other_word,0)+occurrence_count
        yield word, stripe
    
    def reducer_2(self, key, values):
        yield str(key), list(values)[0]
        
if __name__ == '__main__':
    MRbuildStripes.run()

Overwriting buildStripes_stopwords_20.py


In [24]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python buildStripes_stopwords_20.py \
        -r hadoop hdfs://{TEST_20}/* \
        --file ten_thousand_20.json \
        --output-dir={OUTPUT_PATH} \
        --no-output
            



18/03/06 15:53:11 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/buildStripes_stopwords_20.lteo01.20180306.155312.676147
Copying local files to hdfs:///user/lteo01/tmp/mrjob/buildStripes_stopwords_20.lteo01.20180306.155312.676147/files/...
Running step 1 of 2...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob7797187335133637485.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altis

Removing temp directory /tmp/buildStripes_stopwords_20.lteo01.20180306.155312.676147...

real	7m27.141s
user	0m51.520s
sys	0m2.541s


In [25]:
!hadoop fs -cat {OUTPUT_PATH}/* > google_stripes_20

In [26]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python invertedIndex_v2.py \
        -r hadoop google_stripes_20 \
        --output-dir={OUTPUT_PATH} \
        --no-output

18/03/06 16:00:43 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/invertedIndex_v2.lteo01.20180306.160044.358309
Copying local files to hdfs:///user/lteo01/tmp/mrjob/invertedIndex_v2.lteo01.20180306.160044.358309/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob9036766339060416359.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v

In [27]:
!hadoop fs -cat {OUTPUT_PATH}/* > google_index_20

In [28]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python similarity_v2.py \
        -r hadoop google_index_20 \
        --output-dir={OUTPUT_PATH} \
        --no-output

18/03/06 16:01:41 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/similarity_v2.lteo01.20180306.160142.297158
Copying local files to hdfs:///user/lteo01/tmp/mrjob/similarity_v2.lteo01.20180306.160142.297158/files/...
Running step 1 of 2...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob2570539209473584308.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/time

Removing temp directory /tmp/similarity_v2.lteo01.20180306.160142.297158...

real	5m2.989s
user	0m48.174s
sys	0m2.341s


In [29]:
!hadoop fs -cat {OUTPUT_PATH}/* > google_similarities_2
!cat google_similarities_2 | head

1.0	["inspect - traders", 1.0, 1.0, 1.0, 1.0]
1.0	["ceremonies - petitions", 1.0, 1.0, 1.0, 1.0]
1.0	["settlements - stressed", 1.0, 1.0, 1.0, 1.0]
1.0	["conveying - interfering", 1.0, 1.0, 1.0, 1.0]
1.0	["shareholders - stockholders", 1.0, 1.0, 1.0, 1.0]
1.0	["recognizing - waged", 1.0, 1.0, 1.0, 1.0]
1.0	["believer - populace", 1.0, 1.0, 1.0, 1.0]
1.0	["constructing - irrigation", 1.0, 1.0, 1.0, 1.0]
1.0	["letting - ornament", 1.0, 1.0, 1.0, 1.0]
1.0	["applicability - disadvantages", 1.0, 1.0, 1.0, 1.0]
cat: write error: Broken pipe


In [ ]:
############################################
# Pretty print systems tests
# Note: adjust print formatting if you need to
############################################

import json
for i in range(2,3):
    print '—'*110
    print "Systems test ",i," - Similarity measures"
    print '—'*110
    print "{0:>15} |{1:>15} |{2:>15} |{3:>15} |{4:>15} |{5:>15}".format(
    "average", "pair", "cosine", "jaccard", "overlap", "dice")
    print '-'*110

    with open("google_similarities_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            avg,stripe = line.split("\t")
            stripe = json.loads(stripe)
            
            print "{0:>15f} |{1:>15} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(float(avg),
                stripe[0], float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]))

### HW5.7.3 Running the full dataset on Altiscale

Please contact the TAs for approval after obtaining results from 5.7.2. We have ran into issues in the past where the clusters froze because people did not test their code on a smaller dataset.

In [30]:
%%writefile buildStripes_stopwords.py
#!~/opt/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import itertools

class MRbuildStripes(MRJob):
    
    SORT_VALUES = True
    
    def steps(self):
        return [
            MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper,
                reducer=self.reducer,
                jobconf = {
                    "mapreduce.job.reduces": "64",
                    "mapreduce.job.maps": "64",
#                     "SORT_VALUES":True
                }
            ),
            MRStep(
                reducer=self.reducer_2,
                jobconf = {
                    "mapreduce.job.reduces": "1",
                    "SORT_VALUES":True
                }
            )
            
        ]
    
    def mapper_init(self):
        self.idx = 9001  # To define when feature set starts
        self.filename = 'ten_thousand_FULL.json'
        
        self.top_words = []
        self.features = []
        #with open('features_20.json', 'r') as infile:
        #    self.features = json.loads(infile.read())
        with open(self.filename, 'r') as infile:
            self.top_words = json.loads(infile.read())
            self.features =self.top_words[self.idx:]

    def mapper(self, _, line):
        fields = line.lower().strip("\n").split("\t")
        words = fields[0].split(" ")
        occurrence_count = int(fields[1])
        filtered_words = [word.decode('utf-8', 'ignore') for word in words if word.decode('utf-8', 'ignore') in self.top_words]
        for subset in itertools.combinations(sorted(set(filtered_words)), 2):
            if subset[0] in self.top_words and subset[1] in self.features:
                yield subset[0], (subset[1], occurrence_count)
            if subset[1] in self.top_words and subset[0] in self.features:
                yield subset[1], (subset[0], occurrence_count)
    
    def reducer(self, word, occurrence_counts):
        stripe = {}
        for other_word, occurrence_count in occurrence_counts:
            stripe[other_word] = stripe.get(other_word,0)+occurrence_count
        yield word, stripe
    
    def reducer_2(self, key, values):
        yield str(key), list(values)[0]
        
if __name__ == '__main__':
    MRbuildStripes.run()



Overwriting buildStripes_stopwords.py


In [31]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python buildStripes_stopwords.py \
        -r hadoop hdfs://{FULL_DATA}/* \
        --file ten_thousand_FULL.json \
        --output-dir={OUTPUT_PATH} \
        --no-output
            



18/03/06 16:09:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/buildStripes_stopwords.lteo01.20180306.160925.905278
Copying local files to hdfs:///user/lteo01/tmp/mrjob/buildStripes_stopwords.lteo01.20180306.160925.905278/files/...
Running step 1 of 2...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob3862403407545034719.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.c

Removing temp directory /tmp/buildStripes_stopwords.lteo01.20180306.160925.905278...

real	30m3.979s
user	0m55.912s
sys	0m2.750s


In [33]:
!hadoop fs -cat {OUTPUT_PATH}/* > google_stripes_FULL

In [34]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python invertedIndex_v2.py \
        -r hadoop google_stripes_FULL \
        --output-dir={OUTPUT_PATH} \
        --no-output

18/03/06 16:43:40 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/invertedIndex_v2.lteo01.20180306.164340.912983
Copying local files to hdfs:///user/lteo01/tmp/mrjob/invertedIndex_v2.lteo01.20180306.164340.912983/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob6369264445573276562.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v

In [35]:
!hadoop fs -cat {OUTPUT_PATH}/* > google_index_FULL

In [36]:
# Run in Hadoop

OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}

!time python similarity_v2.py \
        -r hadoop google_index_FULL \
        --output-dir={OUTPUT_PATH} \
        --no-output

18/03/06 16:44:47 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/lteo01/.Trash/Current
Using configs in /home/lteo01/.mrjob.conf
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/similarity_v2.lteo01.20180306.164448.602492
Copying local files to hdfs:///user/lteo01/tmp/mrjob/similarity_v2.lteo01.20180306.164448.602492/files/...
Running step 1 of 2...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob6219423789189061735.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/time

  Task Id : attempt_1509050304403_25958_r_000000_1, Status : FAILED
Container [pid=6567,containerID=container_1509050304403_25958_01_000068] is running beyond physical memory limits. Current usage: 2.5 GB of 2.5 GB physical memory used; 4.3 GB of 5.3 GB virtual memory used. Killing container.
Dump of the process-tree for container_1509050304403_25958_01_000068 :
	|- PID PPID PGRPID SESSID CMD_NAME USER_MODE_TIME(MILLIS) SYSTEM_TIME(MILLIS) VMEM_USAGE(BYTES) RSSMEM_USAGE(PAGES) FULL_CMD_LINE
	|- 6574 6567 6567 6567 (java) 6903 561 4572450816 664683 /usr/java/default/bin/java -XX:NewRatio=8 -Djava.net.preferIPv4Stack=true -Xmx2540m -Djava.io.tmpdir=/var/storage/nm-sdh1/nm-local/usercache/lteo01/appcache/application_1509050304403_25958/container_1509050304403_25958_01_000068/tmp -Dlog4j.configuration=container-log4j.properties -Dyarn.app.container.log.dir=/var/storage/nm-sdd1/nm-logs/application_1509050304403_25958/container_1509050304403_25958_01_000068 -Dyarn.app.container.log.filesize=

Removing temp directory /tmp/similarity_v2.lteo01.20180306.164448.602492...

real	11m33.738s
user	0m49.698s
sys	0m2.419s


In [37]:
!hadoop fs -cat {OUTPUT_PATH}/* > google_similarities_3
!cat google_similarities_3 | head

cat: `/user/lteo01/tests/*': No such file or directory


In [ ]:
############################################
# Pretty print systems tests
# Note: adjust print formatting if you need to
############################################

import json
for i in range(2,3):
    print '—'*110
    print "Systems test ",i," - Similarity measures"
    print '—'*110
    print "{0:>15} |{1:>15} |{2:>15} |{3:>15} |{4:>15} |{5:>15}".format(
    "average", "pair", "cosine", "jaccard", "overlap", "dice")
    print '-'*110

    with open("google_similarities_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            avg,stripe = line.split("\t")
            stripe = json.loads(stripe)
            
            print "{0:>15f} |{1:>15} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(float(avg),
                stripe[0], float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]))

#### Pretty print results
NOTE: depending on how you processed the stop words your results may differ from the table provided. 

In [ ]:
print "\nTop/Bottom 20 results - Similarity measures - sorted by cosine"
print "(From the entire data set)"
print '—'*117
print "{0:>30} |{1:>15} |{2:>15} |{3:>15} |{4:>15} |{5:>15}".format(
        "pair", "cosine", "jaccard", "overlap", "dice", "average")
print '-'*117

for stripe in sortedSims[:20]:
    print "{0:>30} |{1:>15f} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(
        stripe[0], float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]), float(stripe[5]) )

print '—'*117

for stripe in sortedSims[-20:]:
    print "{0:>30} |{1:>15f} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(
        stripe[0], float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]), float(stripe[5]) )


In [ ]:
Top/Bottom 20 results - Similarity measures - sorted by cosine
(From the entire data set)
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
                          pair |         cosine |        jaccard |        overlap |           dice |        average
---------------------------------------------------------------------------------------------------------------------
                   cons - pros |       0.894427 |       0.800000 |       1.000000 |       0.888889 |       0.895829
            forties - twenties |       0.816497 |       0.666667 |       1.000000 |       0.800000 |       0.820791
                    own - time |       0.809510 |       0.670563 |       0.921168 |       0.802799 |       0.801010
                 little - time |       0.784197 |       0.630621 |       0.926101 |       0.773473 |       0.778598
                  found - time |       0.783434 |       0.636364 |       0.883788 |       0.777778 |       0.770341
                 nova - scotia |       0.774597 |       0.600000 |       1.000000 |       0.750000 |       0.781149
                   hong - kong |       0.769800 |       0.615385 |       0.888889 |       0.761905 |       0.758995
                   life - time |       0.769666 |       0.608789 |       0.925081 |       0.756829 |       0.765091
                  time - world |       0.755476 |       0.585049 |       0.937500 |       0.738209 |       0.754058
                  means - time |       0.752181 |       0.587117 |       0.902597 |       0.739854 |       0.745437
                   form - time |       0.749943 |       0.588418 |       0.876733 |       0.740885 |       0.738995
       infarction - myocardial |       0.748331 |       0.560000 |       1.000000 |       0.717949 |       0.756570
                 people - time |       0.745788 |       0.573577 |       0.923875 |       0.729010 |       0.743063
                 angeles - los |       0.745499 |       0.586207 |       0.850000 |       0.739130 |       0.730209
                  little - own |       0.739343 |       0.585834 |       0.767296 |       0.738834 |       0.707827
                    life - own |       0.737053 |       0.582217 |       0.778502 |       0.735951 |       0.708430
          anterior - posterior |       0.733388 |       0.576471 |       0.790323 |       0.731343 |       0.707881
                  power - time |       0.719611 |       0.533623 |       0.933586 |       0.695898 |       0.720680
              dearly - install |       0.707107 |       0.500000 |       1.000000 |       0.666667 |       0.718443
                   found - own |       0.704802 |       0.544134 |       0.710949 |       0.704776 |       0.666165
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
           arrival - essential |       0.008258 |       0.004098 |       0.009615 |       0.008163 |       0.007534
         governments - surface |       0.008251 |       0.003534 |       0.014706 |       0.007042 |       0.008383
                king - lesions |       0.008178 |       0.003106 |       0.017857 |       0.006192 |       0.008833
              clinical - stood |       0.008178 |       0.003831 |       0.011905 |       0.007634 |       0.007887
               till - validity |       0.008172 |       0.003367 |       0.015625 |       0.006711 |       0.008469
            evidence - started |       0.008159 |       0.003802 |       0.012048 |       0.007576 |       0.007896
               forces - record |       0.008152 |       0.003876 |       0.011364 |       0.007722 |       0.007778
               primary - stone |       0.008146 |       0.004065 |       0.009091 |       0.008097 |       0.007350
             beneath - federal |       0.008134 |       0.004082 |       0.008403 |       0.008130 |       0.007187
                factors - rose |       0.008113 |       0.004032 |       0.009346 |       0.008032 |       0.007381
           evening - functions |       0.008069 |       0.004049 |       0.008333 |       0.008065 |       0.007129
                   bone - told |       0.008061 |       0.003704 |       0.012346 |       0.007380 |       0.007873
             building - occurs |       0.008002 |       0.003891 |       0.010309 |       0.007752 |       0.007489
                 company - fig |       0.007913 |       0.003257 |       0.015152 |       0.006494 |       0.008204
               chronic - north |       0.007803 |       0.003268 |       0.014493 |       0.006515 |       0.008020
             evaluation - king |       0.007650 |       0.003030 |       0.015625 |       0.006042 |       0.008087
             resulting - stood |       0.007650 |       0.003663 |       0.010417 |       0.007299 |       0.007257
                 agent - round |       0.007515 |       0.003289 |       0.012821 |       0.006557 |       0.007546
         afterwards - analysis |       0.007387 |       0.003521 |       0.010204 |       0.007018 |       0.007032
            posterior - spirit |       0.007156 |       0.002660 |       0.016129 |       0.005305 |       0.007812

# HW5.8 - Evaluation of synonyms that your discovered

In this part of the assignment you will evaluate the success of you synonym detector. Take the top 1,000 closest/most similar/correlative pairs of words as determined by your measure in HW5.7, and use the synonyms function from the wordnet synonnyms list from the nltk package (see provided code below).

For each (word1,word2) pair, check to see if word1 is in the list, 
synonyms(word2), and vice-versa. If one of the two is a synonym of the other, 
then consider this pair a 'hit', and then report the precision, recall, and F1 measure  of 
your detector across your 1,000 best guesses. Report the macro averages of these measures.

### Calculate performance measures:
$$Precision (P) = \frac{TP}{TP + FP} $$  
$$Recall (R) = \frac{TP}{TP + FN} $$  
$$F1 = \frac{2 * ( precision * recall )}{precision + recall}$$


We calculate Precision by counting the number of hits and dividing by the number of occurances in our top1000 (opportunities)   
We calculate Recall by counting the number of hits, and dividing by the number of synonyms in wordnet (syns)


Other diagnostic measures not implemented here:  https://en.wikipedia.org/wiki/F1_score#Diagnostic_Testing

In [ ]:
''' Performance measures '''
from __future__ import division
import numpy as np
import json
import nltk
from nltk.corpus import wordnet as wn
import sys
#print all the synset element of an element
def synonyms(string):
    syndict = {}
    for i,j in enumerate(wn.synsets(string)):
        syns = j.lemma_names()
        for syn in syns:
            syndict.setdefault(syn,1)
    return syndict.keys()
hits = []

TP = 0
FP = 0

TOTAL = 0
flag = False # so we don't double count, but at the same time don't miss hits

top1000sims = []
with open("sims2/top1000sims","r") as f:
    for line in f.readlines():

        line = line.strip()
        avg,lisst = line.split("\t")
        lisst = json.loads(lisst)
        lisst.append(avg)
        top1000sims.append(lisst)
    

measures = {}
not_in_wordnet = []

for line in top1000sims:
    TOTAL += 1

    pair = line[0]
    words = pair.split(" - ")
    
    for word in words:
        if word not in measures:
            measures[word] = {"syns":0,"opps": 0,"hits":0}
        measures[word]["opps"] += 1 
    
    syns0 = synonyms(words[0])
    measures[words[1]]["syns"] = len(syns0)
    if len(syns0) == 0:
        not_in_wordnet.append(words[0])
        
    if words[1] in syns0:
        TP += 1
        hits.append(line)
        flag = True
        measures[words[1]]["hits"] += 1
        
        
        
    syns1 = synonyms(words[1]) 
    measures[words[0]]["syns"] = len(syns1)
    if len(syns1) == 0:
        not_in_wordnet.append(words[1])

    if words[0] in syns1:
        if flag == False:
            TP += 1
            hits.append(line)
            measures[words[0]]["hits"] += 1
            
    flag = False    

precision = []
recall = []
f1 = []

for key in measures:
    p,r,f = 0,0,0
    if measures[key]["hits"] > 0 and measures[key]["syns"] > 0:
        p = measures[key]["hits"]/measures[key]["opps"]
        r = measures[key]["hits"]/measures[key]["syns"]
        f = 2 * (p*r)/(p+r)
    
    # For calculating measures, only take into account words that have synonyms in wordnet
    if measures[key]["syns"] > 0:
        precision.append(p)
        recall.append(r)
        f1.append(f)

    
# Take the mean of each measure    
print "—"*110    
print "Number of Hits:",TP, "out of top",TOTAL
print "Number of words without synonyms:",len(not_in_wordnet)
print "—"*110 
print "Precision\t", np.mean(precision)
print "Recall\t\t", np.mean(recall)
print "F1\t\t", np.mean(f1)
print "—"*110  

print "Words without synonyms:"
print "-"*100

for word in not_in_wordnet:
    print synonyms(word),word

    

### Sample output

In [ ]:
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Number of Hits: 31 out of top 1000
Number of words without synonyms: 67
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Precision	0.0280214404967
Recall		0.0178598869579
F1		0.013965517619
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Words without synonyms:
----------------------------------------------------------------------------------------------------
[] scotia
[] hong
[] kong
[] angeles
[] los
[] nor
[] themselves
[] 
.......

# HW5.9 - OPTIONAL: using different vocabulary subsets

Repeat HW5 using vocabulary words ranked from 8001,-10,000;  7001,-10,000; 6001,-10,000; 5001,-10,000; 3001,-10,000; and 1001,-10,000;
Dont forget to report you Cluster configuration.

Generate the following graphs:
-- vocabulary size (X-Axis) versus CPU time for indexing
-- vocabulary size (X-Axis) versus number of pairs processed
-- vocabulary size (X-Axis) versus F1 measure, Precision, Recall

# HW5.10  - OPTIONAL 

There are many good ways to build our synonym detectors, so for this optional homework, 
measure co-occurrence by (left/right/all) consecutive words only, 
or make stripes according to word co-occurrences with the accompanying 
2-, 3-, or 4-grams (note here that your output will no longer 
be interpretable as a network) inside of the 5-grams.

# HW5.11 - OPTIONAL 

Once again, benchmark your top 10,000 associations (as in 5.7), this time for your
results from 5.8. Has your detector improved?